# Setup

You'll want to set file paths here.

For Xenium data, 

```
wget https://cf.10xgenomics.com/samples/xenium/1.6.0/Xenium_V1_hColon_Non_diseased_Base_FFPE/Xenium_V1_hColon_Non_diseased_Base_FFPE_outs.zip
```

If you need to unzip the spatial/cells file:

`unzip Xenium_V1_hColon_Non_diseased_Base_FFPE_outs.zip Xenium_V1_hColon_Non_diseased_Base_FFPE_outs`

`tar -xvf Xenium_V1_hColon_Non_diseased_Base_FFPE_outs/cells.csv.gz`

(replacing <file_path_spatial> and <data_dir> with the paths to the cells.csv.gz file and the data directory).


In [ ]:
%load_ext autoreload
%autoreload 2

from crispr import Spatial
import scanpy as sc
import squidpy as sq
import os
import pandas as pd

ddd = os.path.join(os.path.expanduser("~"), str(
    "projects/data/public/xenium/Xenium_V1_hColon_Non_diseased_Base_FFPE_outs"
    ))
fpd, fps = "cell_feature_matrix.h5", "cells.csv"
file_path, file_path_spatial = [os.path.join(ddd, x) for x in [fpd, fps]]

genes = ["CDKN1A", "CDKN2A", "TP53", "PLAUR", 
         "PTGER4", "ATG16L1", "NOD2", "LACC1"]

pd.options.display.max_columns = 100


# Object

In [ ]:
self = Spatial(file_path, file_path_spatial=file_path_spatial, 
               visium=False, col_cell_type="majority_voting")

# Processing Data

## Preprocessing

In [ ]:
self.preprocess(regress_out=None)

## Clustering

In [ ]:
_ = self.cluster()

## Annotation

In [ ]:
_ = self.annotate_clusters("Cells_Intestinal_Tract.pkl")

# Analysis

You can run everything in a one-liner command:
```
self.analyze_spatial(genes=genes, method_autocorr="moran", alpha=0.005, 
                     n_perms=10, seed=1618, figsize_multiplier=2)
```

You can also run the steps of the overall workflow separately. Here are some examples running parts of the workflow (sometimes illustrating further customization options). 

## Basic Plots

In [ ]:
_ = self.plot(color=genes, shape="circle")
_ = self.plot(color="predicted_labels")

In [ ]:
anchor_key = 192
adata = self.adata.copy()
col_cell_type = None

if not col_cell_type:
    col_cell_type = self._columns["col_cell_type"]
sq.tl.var_by_distance(adata, genes, col_cell_type, spatial_key=self._assay_spatial)
sq.pl.var_by_distance(adata, genes, anchor_key)

## Centrality Scores 

In [ ]:
self.calculate_centrality()  # run

## Co-Occurrence of Cell Types

We'll first randomly choose three clusters to focus the plots on, then run the analysis.


In [ ]:
kct = list(self.rna.obs[self._columns["col_cell_type"]].sample(3))
_ = self.find_cooccurrence(key_cell_type=kct, figsize=(40, 20),
                           kws_plot=dict(wspace=3, palette="tab20"))

## Neighbors Enrichment Analysis

In [ ]:
_ = self.calculate_neighborhood()  # analyze enrichment

## Spatially-Variable Genes

In [ ]:
self.find_svgs(genes=genes, method="moran", n_perms=10)

## Receptor-Ligand Interactions

In [ ]:
out = self.calculate_receptor_ligand(
    key_sources=None, key_targets=None, p_threshold=0.0005, 
    remove_ns=True, hspace=0.2)